In [8]:
#all_beauty
import gzip
import json

def parse(path):
  res = []
  g = gzip.open(path, 'rb')
  for l in g:
    res.append(json.loads(l))
  return res

reviews = parse('/content/All_Beauty_5.json.gz')

In [9]:
reviews[0]

{'overall': 5.0,
 'verified': True,
 'reviewTime': '09 1, 2016',
 'reviewerID': 'A3CIUOJXQ5VDQ2',
 'asin': 'B0000530HU',
 'style': {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice Blue'},
 'reviewerName': 'Shelly F',
 'reviewText': 'As advertised. Reasonably priced',
 'summary': 'Five Stars',
 'unixReviewTime': 1472688000}

##**П.1**

1. Выделить ключевые слова (например, при помощи keyBert): 1-, 2-граммы. Среди ключевых слов нас интересуют лишь существительные и NE. Если общий словарь получился не очень большой, можно отфильтровать названия вручную, если нет, то сначала можно выделить NE и оставить их, для остальных ключевых слов сделать отсечку по частотности. 

Плюсы: полнота. \
Минусы: вероятно, ручная сортировка; много вычислений, точность -- погрешности из-за поиска ключевых слов, не учитывание контекста. 

2. В metadata есть название товара с сайта, а названии наверняка содержится необходимая нам NE, поэтому можно попробовать выделить именованную сущность. Либо эта сущность будет названием бренда, тогда можно посмотреть на соседнее слово-существительное (например, [MAC] [lipstick]), либо оно включает в себя название бренда (например, [Xiaomi Note 5]). Бренд также содержится в отдельной графе metadata. В словарь можно добавить либо целиком название модели (второй случай), либо коллокацию бренд-предмет и предмет ("lipstick", "MAC lipstick"), и искать в отзывах совпадения с этим. 

Плюсы: точность, простота вычислений. \
Минусы: полнота, много памяти (один объект может содержаться несколько раз, как в примере с помадой). 

Дополнительно к отзывам нужно будет парсить Metadata. 

3. В названии товара точно будет содержаться слово, относящееся к нему. Составим словарь дескрипторов (либо вручную, либо отсортируя по частотности ключевые слова), затем составим словарь для вообще всех слов в названиях товаров (предварительно предобработав их).
Составим матрицу размером (словарь * словарь дескрипторов). Для каждого слова в словаре посчитаем косинусную близость со всеми дескрипторами, выберем из них максимальную. Взяв максимальные значения, получим вектор, длина которого будет равна количеству слов в словаре. Теперь можно либо искать в названии товара самое похожее на дескриптор слово и добавлять его в словарь, либо установить какое-то пороговое значение и добавлять все слова со значением выше порогового. Можно проделывать это только с существительными. 

Плюсы: скорость вычислений, простота. \
Минусы: точность, полнота -- сложно подобрать пороговое значение, может попасть либо очень много лишнего, либо потеряться много полезного; не понятно, что делать с NE типа названия брендов или моделей. 

Дополительно потребуется парсить Metadata и пользоваться предобработанными эмбеддингами. 

##**П.2**

In [10]:
!pip3 install keybert
!pip install spacy
!pip install stanza

import spacy
from keybert import KeyBERT
import stanza
from collections import defaultdict

NER = stanza.Pipeline("en", processors="tokenize,ner")
pos_tagger = spacy.load("en_core_web_sm")
kw_model = KeyBERT('clips/mfaq')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.8 MB/s 
     |████████████████████████████████| 237 kB 44.7 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
     |████████████████████████████████| 5.8 MB 52.7 MB/s 
     |████████████████████████████████| 1.3 MB 69.5 MB/s 
     |████████████████████████████████| 182 kB 71.0 MB/s 
     |████████████████████████████████| 7.6 MB 63.7 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=e578b08134c026ef388a84a228adb1cf9c9a1eb568c635ef9dc2e5fbb9fa0d8e
  Stored in directory: /root/.cache/pip/wheels/6c/bc/8b/a51bee77aec33895e6c8c236144b4cc10875659c4d2c80f070
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1b86af17caefd12981c84e478bd1694da219e22b745165c5e1007be1e63cb054
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [11]:
def get_NEs_from_review(review):
  NEs = []
  cur_ents = []

  entities = NER(review)
  for entity in entities.ents:

    if entity.type == 'PRODUCT':
      cur_ents.extend(entity.text.lower().split())

      if entity.text.lower() not in NEs:
        NEs.append(entity.text.lower())


  tags = defaultdict(str)
  pos_tags = pos_tagger(review)
  for token in pos_tags:
    tags[token.text.lower()] = token.pos_ #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

  kwords = kw_model.extract_keywords(review, stop_words=['amazon']+cur_ents)
  for kword, freq in kwords:
    #print(kword, freq)
    if tags[kword] == 'NOUN' and kword not in NEs:
      NEs.append(kword)

  return NEs



In [20]:
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/gdrive')

#with open('/content/gdrive/My Drive/brand_model_names', 'w') as f:
#  for review in tqdm(reviews):

#    if 'reviewText' in review.keys():
#      entities = NER(review['reviewText'])
#      for entity in entities.ents:

#        if entity.type == 'PRODUCT':
#          f.write(entity.text.lower()+'\n')

#with open('/content/gdrive/My Drive/NEs', 'w') as f:
#  for review in tqdm(reviews):
#    if 'reviewText' in review.keys():
#      for NE in get_NEs_from_review(review['reviewText']):
#        f.write(NE+'\n')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


100%|██████████| 5269/5269 [37:21<00:00,  2.35it/s]


In [172]:
import re
from tqdm import tqdm
from pprint import pprint
from collections import defaultdict

NEs = defaultdict(int)
names = defaultdict(int)

with open('/content/NEs (2)', 'r') as f:
  for line in f:
    NE = line.strip() #''.join(re.findall(r'[\w ]', line.strip()))
    NEs[NE] += 1

with open('/content/brand_model_names', 'r') as f:
  for line in f:
    name = line.strip() #''.join(re.findall(r'[\w ]', line.strip()))
    names[name] += 1

In [173]:
sorted(list(NEs.items()), key = lambda x: x[1])

[('soothing', 1),
 ('lemon', 1),
 ('sugar', 1),
 ('invigorating clean mint"', 1),
 ('aftertaste', 1),
 ('crest pro-health', 1),
 ('scope', 1),
 ('yellow #5', 1),
 ('yellow #6', 1),
 ('invigorating clean', 1),
 ('burn', 1),
 ('mint', 1),
 ('crest mouthwashes', 1),
 ('children', 1),
 ('risk', 1),
 ('norelco', 1),
 ('the arcitec 1090', 1),
 ('quadra action', 1),
 ('smarttouch xl', 1),
 ('hairs', 1),
 ('redness', 1),
 ('sanitizer', 1),
 ('this clean & clear cleanser', 1),
 ('result', 1),
 ('mascara', 1),
 ('lipstick', 1),
 ('makeup', 1),
 ('exfoliating', 1),
 ('glide comfort plus unflavored', 1),
 ('spoiler gillette', 1),
 ('merkur platinum', 1),
 ('blade', 1),
 ('benefits', 1),
 ('exfoliator', 1),
 ('lactobacillus pumpkin ferment extract', 1),
 ('ceteareth-20', 1),
 ('clarisonic', 1),
 ('samples', 1),
 ('sampler', 1),
 ('scar', 1),
 ('goop', 1),
 ('neosporin', 1),
 ('eczema', 1),
 ('eyelash', 1),
 ('lash', 1),
 ('sting', 1),
 ('trip', 1),
 ('moisturization', 1),
 ('the mad hippie face cre

In [174]:
min_freq_treshold = 10

print(len(NEs), len(names))
NEs_keys = list(NEs.keys())

for NE in NEs_keys:
  if NE not in names.keys() and NEs[NE] < min_freq_treshold:
    del NEs[NE]

print(len(NEs))

667 133
271


##**П.3**

In [175]:
from sklearn.linear_model import ElasticNet
# я не смогла придумать, как с помощью биграмм в НЛТК можно учесть NE+сосед в том случае, когда NE состоит более чем из одного слова
# такие штуки я тоже считаю биграммами, потому что хочется, чтобы сущность NE была однородной
# такие биграммы тоже учитываются в общем количестве биграмм, добавляются отдельно

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words('english'))

NEs_left_neighbor_compiled = {NE: re.compile(NE+' '+r'[\w#&-]+') for NE in NEs.keys()}
NEs_right_neighbor_compiled = {NE: re.compile(r'[\w#&-]+'+' '+NE) for NE in NEs.keys()}
NEs_long_compiled = {NE: re.compile(NE) for NE in NEs.keys() if len(NE.split())>1}
word_pattern = re.compile(r'[\w#&-]+')

words = {}
words_total = 0
bigrams = {} # (bigram, NE, neighbour)
bms = [] # посто биграммы без отдельных пометок, чтобы соранять только уникальные
bigrams_total = 0

for review in tqdm(reviews):
  if 'reviewText' in review.keys():
    tokenized = [t for t in re.findall(word_pattern, review['reviewText'].lower()) if t not in stops]
    bigrams_total += len(tokenized)-1
    words_total += len(tokenized)

    for token in tokenized:
      if token not in words.keys():
        words[token] = 0
      words[token] += 1

    tokenized = ' '.join(tokenized)

    for NE in NEs.keys():

      if len(NE.split()) > 1:
        for long_NE in re.findall(NEs_long_compiled[NE], tokenized):
          if NE not in words.keys():
            words[NE] = 0
          words[NE] += 1

      b = [] # все биграммы с текущей NE, которые нашлись в отзыве
      for bigr in re.findall(NEs_left_neighbor_compiled[NE], tokenized):
        a = bigr
        w1, w2 = bigr.rsplit(maxsplit = 1)
        b.append((bigr, w1, w2))

      for bigr in re.findall(NEs_right_neighbor_compiled[NE], tokenized):
         w1, w2 = bigr.split(maxsplit = 1)
         b.append((bigr, w2, w1))

      if len(NE.split()) > 1:
        bigrams_total += len(b)

      for bigr in b:

        if bigr in bigrams.keys():
          bigrams[bigr] += 1
        elif bigr[0] in bms:
          bigrams[(bigr[0], bigr[2], bigr[1])] += 1
        else:
          bms.append(bigr[0])
          bigrams[bigr] = 1

# учитывать длинные NE за одну сущность, то есть как одно слово
words_total += len(NEs_long_compiled)          

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 5269/5269 [00:17<00:00, 302.74it/s]


##**П.4**

In [176]:
# t-score
from math import sqrt, log
def tscore_count(bigr):
  x = bigrams[bigr]/bigrams_total
  mu = (words[bigr[1]]/words_total)*(words[bigr[2]]/words_total)
  stddev = bigrams[bigr]/bigrams_total

  return (x - mu)/sqrt(stddev/words_total)

tscore_bigrams = []
for bigr in list(bigrams.keys()):
  if bigrams[bigr] != 0 and bigr[1] in words.keys() and bigr[2] in words.keys():
    tscore_bigrams.append((bigr, tscore_count(bigr)))

pprint(sorted(tscore_bigrams, key = lambda x: -x[-1])[:30])

[(('body wash', 'body', 'wash'), 27.57346193575953),
 (('shower gel', 'gel', 'shower'), 18.785220378613104),
 (('love product', 'product', 'love'), 15.922679576544946),
 (('great product', 'product', 'great'), 14.610360265836713),
 (('shampoo conditioner', 'shampoo', 'conditioner'), 14.529604119854161),
 (('smells good', 'smells', 'good'), 14.298289834397977),
 (('love shampoo', 'shampoo', 'love'), 13.00603459330929),
 (('hair feel', 'feel', 'hair'), 12.826474692968883),
 (('love smell', 'smell', 'love'), 12.307923800606071),
 (('skin feel', 'feel', 'skin'), 12.142272120471933),
 (('eye cream', 'cream', 'eye'), 12.037815726830397),
 (('love scent', 'scent', 'love'), 11.732357395267035),
 (('body lotion', 'body', 'lotion'), 11.21334990597848),
 (('smells great', 'smells', 'great'), 11.103660562184164),
 (('shea butter', 'butter', 'shea'), 10.728387619126215),
 (('leaves skin', 'skin', 'leaves'), 10.51078716479233),
 (('shampoo condition', 'condition', 'shampoo'), 10.34168729850529),
 ((

In [177]:
# PMI
def PMI_count(bigr):
  # frac = (bigrams[bigr]/bigrams_total)/((words[bigr[1]]/words_total)*(words[bigr[2]]/words_total))
  frac = bigrams[bigr] * words_total**2 / (words[bigr[1]] * words[bigr[2]] * bigrams_total)
  return log(frac, 2)

PMI_bigrams = []
for bigr in list(bigrams.keys()):
  if bigr[1] in words.keys() and bigr[2] in words.keys():
    PMI_bigrams.append((bigr, PMI_count(bigr)))

pprint(sorted(PMI_bigrams, key = lambda x: -x[-1])[:30])

[(('quadra action smarttouch', 'quadra action', 'smarttouch'),
  16.703923988348137),
 (('lactobacillus pumpkin ferment extract ceteareth-20',
   'lactobacillus pumpkin ferment extract',
   'ceteareth-20'),
  16.703923988348137),
 (('blown aerosol foam conditioner', 'aerosol foam conditioner', 'blown'),
  16.703923988348137),
 (('rotation pantene volume', 'pantene volume', 'rotation'),
  16.703923988348137),
 (('sudzing girley', 'girley', 'sudzing'), 16.703923988348137),
 (('braun 320s-4', 'braun', '320s-4'), 16.11896148762698),
 (('crest mouthwashes tens', 'crest mouthwashes', 'tens'), 15.703923988348135),
 (('ceteareth-20 cetyl', 'ceteareth-20', 'cetyl'), 15.703923988348135),
 (('held wp-450', 'wp-450', 'held'), 15.118961487626978),
 (('aerosol foam conditioner unlike', 'aerosol foam conditioner', 'unlike'),
  15.118961487626978),
 (('fekkai foam conditioner unusual', 'fekkai foam conditioner', 'unusual'),
  14.703923988348135),
 (('production filigree', 'filigree', 'production'), 14

In [178]:
bigram_stat = defaultdict(int)
for key in bigrams.keys():
  bigram_stat[bigrams[key]] += 1

for freq in sorted(list(bigram_stat.keys())):
  print('frequency: %s'%freq + '\t' + 'bigrams_with_freq: %s'%bigram_stat[freq])

frequency: 1	bigrams_with_freq: 3943
frequency: 2	bigrams_with_freq: 2643
frequency: 3	bigrams_with_freq: 337
frequency: 4	bigrams_with_freq: 304
frequency: 5	bigrams_with_freq: 2819
frequency: 6	bigrams_with_freq: 822
frequency: 7	bigrams_with_freq: 150
frequency: 8	bigrams_with_freq: 85
frequency: 9	bigrams_with_freq: 38
frequency: 10	bigrams_with_freq: 466
frequency: 11	bigrams_with_freq: 106
frequency: 12	bigrams_with_freq: 115
frequency: 13	bigrams_with_freq: 29
frequency: 14	bigrams_with_freq: 48
frequency: 15	bigrams_with_freq: 51
frequency: 16	bigrams_with_freq: 41
frequency: 17	bigrams_with_freq: 16
frequency: 18	bigrams_with_freq: 18
frequency: 19	bigrams_with_freq: 11
frequency: 20	bigrams_with_freq: 49
frequency: 21	bigrams_with_freq: 14
frequency: 22	bigrams_with_freq: 28
frequency: 23	bigrams_with_freq: 7
frequency: 24	bigrams_with_freq: 9
frequency: 25	bigrams_with_freq: 7
frequency: 26	bigrams_with_freq: 12
frequency: 27	bigrams_with_freq: 5
frequency: 28	bigrams_with_f

In [179]:
# вроде тут хорошо было бы отфильтровать по частотности
# PMI

PMI_bigrams_filtered = []
for bigr in list(bigrams.keys()):
  if bigrams[bigr] > 9 and bigr[1] in words.keys() and bigr[2] in words.keys():
    PMI_bigrams_filtered.append((bigr, PMI_count(bigr)))

pprint(sorted(PMI_bigrams_filtered, key = lambda x: -x[-1])[:30])

[(('lashes curl', 'curl', 'lashes'), 12.50752677554463),
 (('naturally curl', 'curl', 'naturally'), 11.840102114631502),
 (('popsugar box', 'box', 'popsugar'), 11.773186650785249),
 (('shea butter', 'butter', 'shea'), 11.554176868843452),
 (('crest pro-health', 'crest', 'pro-health'), 11.533998986905821),
 (('matrix biolage', 'biolage', 'matrix'), 11.060067798573408),
 (('controls frizz', 'frizz', 'controls'), 11.031498646376638),
 (('speedy delivery', 'delivery', 'speedy'), 11.031498646376638),
 (('foot file', 'foot', 'file'), 10.922564274823474),
 (('shea butter enriched', 'shea butter', 'enriched'), 10.922564274823474),
 (('electric razor', 'razor', 'electric'), 10.833984528912508),
 (('prell condition', 'condition', 'prell'), 10.821280938986293),
 (('32oz size', 'size', '32oz'), 10.715239301575968),
 (('calibra eye', 'eye', 'calibra'), 10.60438831479722),
 (('value https', 'value', 'https'), 10.598888517649845),
 (('butter enriched', 'butter', 'enriched'), 10.554176868843452),
 (('

In [181]:
# Log-likelihood
def log_lhood_count(bigr):
  p = words[bigr[2]]/words_total
  p1 = bigrams[bigr]/words[bigr[1]]
  if p1 == 1:
    p1 -= 10 ** -10
  p2 = max((words[bigr[2]]-bigrams[bigr]), 1)/(words_total - words[bigr[1]])
  if p2 == 1:
    p2 -= 10 ** -10

  #l1 = p**bigrams[bigr] * (1-p)**(words[bigr[1]]-bigrams[bigr])
  #l3 = p1**bigrams[bigr] * (1-p1)**(words[bigr[1]]-bigrams[bigr])
  l1_3 = (p/p1)**bigrams[bigr] * ((1-p)/(1-p1))**(words[bigr[1]]-bigrams[bigr])

  # l2 = p**(words[bigr[2]]-bigrams[bigr[0]]) * (1-p)**(words_total - words[bigr[1]] - words[bigr[2]] + bigrams[bigr[0]])
  # l4 = p2**(words[bigr[2]]-bigrams[bigr[0]]) * (1-p2)**(words_total - words[bigr[1]] - words[bigr[2]] + bigrams[bigr[0]])
  l2_4 = (p/p2)**(words[bigr[2]]-bigrams[bigr]) * ((1-p)/(1-p2))**(words_total - words[bigr[1]] - words[bigr[2]] + bigrams[bigr])

  # по отдельности l-ки занулялись, поэтому сразу считается частное
  # формулу взяла отсюда: http://pioneer.chula.ac.th/~awirote/colloc/statmethod1.htm
  return l1_3*l2_4

log_bigrams = []
for bigr in list(bigrams.keys()):
  if bigr[1] in words.keys() and bigr[2] in words.keys():
    log_bigrams.append((bigr, log_lhood_count(bigr)))

pprint(sorted(log_bigrams, key = lambda x: -x[-1])[:30])

[(('one body', 'body', 'one'), 0.9999999522011204),
 (('body one', 'body', 'one'), 0.9999999522011204),
 (('fine scent', 'scent', 'fine'), 0.9999993138417972),
 (('time without', 'time', 'without'), 0.999998882093231),
 (('leave shampoo', 'shampoo', 'leave'), 0.9999943032975556),
 (('one color', 'color', 'one'), 0.9999907109309114),
 (('soap made', 'soap', 'made'), 0.9999810014215947),
 (('scent conditioner', 'scent', 'conditioner'), 0.9999797756790394),
 (('fragrance hard', 'fragrance', 'hard'), 0.9999468830769522),
 (('skin pretty', 'skin', 'pretty'), 0.9999462779178172),
 (('without wash', 'wash', 'without'), 0.9999345381162869),
 (('love natural', 'love', 'natural'), 0.9999056786434578),
 (('products ever', 'products', 'ever'), 0.999905455521995),
 (('bar would', 'bar', 'would'), 0.9999033368935828),
 (('price shower', 'price', 'shower'), 0.9998791123340667),
 (('clean water', 'water', 'clean'), 0.9998730387369523),
 (('love refreshing', 'love', 'refreshing'), 0.9998463734434233),


In [184]:
# Log-likelihood

log_filtered = []
for bigr in list(bigrams.keys()):
  if bigrams[bigr] > 9 and bigr[1] in words.keys() and bigr[2] in words.keys():
    log_filtered.append((bigr, log_lhood_count(bigr)))

pprint(sorted(log_filtered, key = lambda x: -x[-1])[:30])

[(('skin hair', 'skin', 'hair'), 0.988095537645801),
 (('great hair', 'hair', 'great'), 0.9495546720872232),
 (('scent great', 'scent', 'great'), 0.9464434253068903),
 (('body love', 'body', 'love'), 0.9216080992019666),
 (('product shampoo', 'shampoo', 'product'), 0.8503464201830674),
 (('good use', 'use', 'good'), 0.8318810742540694),
 (('like shampoo', 'shampoo', 'like'), 0.8173927525438802),
 (('well love', 'love', 'well'), 0.8107001164873586),
 (('soap love', 'soap', 'love'), 0.8076621542098092),
 (('love wash', 'wash', 'love'), 0.7794687200119312),
 (('scent love', 'scent', 'love'), 0.7682528443435368),
 (('product wash', 'product', 'wash'), 0.7458965318563551),
 (('use love', 'use', 'love'), 0.7378845367969895),
 (('skin good', 'skin', 'good'), 0.7318917151814196),
 (('shampoo love', 'shampoo', 'love'), 0.7238348575210273),
 (('skin love', 'love', 'skin'), 0.7042951991237207),
 (('good love', 'love', 'good'), 0.7026431834617854),
 (('shower product', 'product', 'shower'), 0.6697

Результаты в принципе получились не очень точные, но лучше всех, на мой взгяд, ранжирует log-likelihood с фильтрацией по частоте биграмм

##**П.5**

In [189]:
grouped_by_NE = {}

for bigr in log_filtered:
  if bigr[0][1] not in grouped_by_NE.keys():
    grouped_by_NE[bigr[0][1]] = [(bigr[0][0], bigr[1])]
  else:
    grouped_by_NE[bigr[0][1]] += [(bigr[0][0], bigr[1])]

for NE in ['perfume', 'cream', 'shampoo', 'fragrance', 'soap']:
    print(NE)
    print('-----')
    pprint(sorted(grouped_by_NE[NE], key = lambda x: -x[1]))
    print()

perfume
-----
[('love perfume', 0.0005851494079413604),
 ('perfume love', 0.0005851494079413604),
 ('use perfume', 7.51135734607905e-06),
 ('perfume use', 7.51135734607905e-06),
 ('shampoo perfume', 1.2199748088943813e-06),
 ('price perfume', 3.584974556356349e-09),
 ('lotion perfume', 1.1500946115843514e-10),
 ('perfume bottle', 6.146429142542883e-11),
 ('perfume shower', 5.1178358974068024e-11),
 ('perfume smell', 3.76615111802604e-12),
 ('perfume light', 2.2967621978564197e-12),
 ('light perfume', 2.2967621978564197e-12),
 ('perfume wanted', 7.116176805278711e-16),
 ('perfume cleans', 3.874592208993987e-16),
 ('bottles perfume', 1.5840098414812077e-19),
 ('perfume husband', 1.2122149807310336e-19),
 ('perfume tube', 6.732838577498149e-22),
 ('wonder perfume', 2.1321262953734193e-23),
 ('spray perfume', 2.5541957882628816e-41)]

cream
-----
[('love cream', 0.05198644158892705),
 ('cream great', 0.018871629239137468),
 ('cream smells', 1.3235308471667334e-05),
 ('products cream', 3.03